# Step 4: SFT - 指令微调

## 学习目标

1. 理解 SFT 和预训练的区别
2. **动手实现**只计算 Assistant 部分的 Loss
3. 理解 LoRA 高效微调的原理

## 核心问题

预训练模型会"续写"，但不会"对话"。如何让它学会遵循指令？

```
预训练: "今天天气" → "真好，适合出游..."（续写）
SFT 后: "今天天气怎么样？" → "今天天气晴朗。"（回答）
```

---

## 1. SFT 数据格式

SFT 使用对话格式的数据：

```json
{
  "conversations": [
    {"role": "system", "content": "你是一个助手。"},
    {"role": "user", "content": "什么是AI？"},
    {"role": "assistant", "content": "AI是人工智能..."}
  ]
}
```

---

## 2. 关键：只计算 Assistant 部分的 Loss

### 为什么？

- 我们希望模型学会**如何回答**，而不是学会**如何提问**
- User 的问题是输入上下文，不应该学习
- 只有 Assistant 的回答需要优化

### 实现方式

```python
# 输入: [system_tokens, user_tokens, assistant_tokens]
# 标签: [-100, -100, ..., -100, assistant_tokens]
#       ↑ 不计算 loss    ↑ 计算 loss

# PyTorch 的 CrossEntropyLoss 会忽略 -100
loss = F.cross_entropy(logits, labels, ignore_index=-100)
```

In [ ]:
import torch
import torch.nn.functional as F

# 演示 ignore_index 的作用
logits = torch.randn(5, 10)  # 5 个位置，10 个类别
labels = torch.tensor([2, -100, -100, 5, 3])  # 只有位置 0, 3, 4 计算 loss

loss = F.cross_entropy(logits, labels, ignore_index=-100)
print(f"Loss: {loss.item():.4f}")
print(f"只计算了 3 个位置的 loss（-100 被忽略）")

### 2.1 练习：实现 _format_conversation

去 `data_exercise.py`，完成 **TODO 1**：

构建 labels，使得只有 Assistant 部分计算 loss

In [ ]:
# 测试你的实现
import importlib
import data_exercise
importlib.reload(data_exercise)

data_exercise.test_sft_dataset()

---

## 3. LoRA：高效微调

### 问题

全参数微调需要：
- 大量显存（存储梯度和优化器状态）
- 每个任务都要保存完整模型

### LoRA 的解决方案

不训练原始权重，而是训练一个低秩"旁路"：

```
原始:  y = Wx           (W 是冻结的)
LoRA:  y = Wx + BAx     (只训练 B 和 A)

W: [n, m]  → n×m 个参数（冻结）
B: [n, r]  → n×r 个参数（训练）
A: [r, m]  → r×m 个参数（训练）

当 r << n, m 时，参数量大大减少
例如: n=m=4096, r=8 → 参数减少 99.6%
```

In [ ]:
import torch.nn as nn

# LoRA 原理演示
n, m, r = 4096, 4096, 8

# 原始参数量
original_params = n * m
print(f"原始参数量: {original_params:,} ({original_params/1e6:.1f}M)")

# LoRA 参数量
lora_params = n * r + r * m
print(f"LoRA 参数量: {lora_params:,} ({lora_params/1e3:.1f}K)")

print(f"\n参数减少: {100 * (1 - lora_params / original_params):.2f}%")

---

## 4. 验证清单

完成本步骤后，你应该能够：

- [ ] 解释 SFT 和预训练的区别
- [ ] 解释为什么只计算 Assistant 部分的 Loss
- [ ] 实现 labels 的构建逻辑
- [ ] 解释 LoRA 的原理和优势

---

## 下一步

进入 [Step 5: RLHF](../step5_rlhf/)，学习如何让模型输出更符合人类偏好。